In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import os

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_doctor_info(html_content):
    """
    Doktor bilgilerini HTML içeriğinden çıkarır
    
    Args:
        html_content (str): HTML sayfası içeriği
        
    Returns:
        list: Doktor bilgilerini içeren sözlüklerin listesi
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    doctors_list = []
    
    # Tüm doktor kartlarını bul
    doctor_cards = soup.find_all('div', class_='dp-doctor-card')
    
    for card in doctor_cards:
        doctor_info = {}
        
        # Doktor adı
        name_element = card.find('span', attrs={'data-tracking-id': 'result-card-name'})
        if name_element:
            doctor_info['name'] = name_element.text.strip()
        
        # Profil resmi
        img_element = card.find('img')
        if img_element:
            doctor_info['profile_image'] = img_element.get('src')
        
        # Branş bilgisi
        specialization = card.find('span', attrs={'data-test-id': 'doctor-specializations'})
        if specialization:
            doctor_info['specialization'] = specialization.text.strip()
        
        # Değerlendirme sayısı
        reviews = card.find('span', class_='opinion-numeral')
        if reviews:
            doctor_info['reviews'] = reviews.text.strip()
        
        # Profil URL'si
        profile_url = card.find('a', attrs={'data-id': 'address-context-cta'})
        if profile_url:
            doctor_info['profile_url'] = profile_url.get('href')
        
        # Doktor onaylı profil mi?
        verified = card.find('span', attrs={'title': 'Teyit edilmiş profil'})
        doctor_info['verified_profile'] = bool(verified)
        
        # Adres bilgisi ve diğer detaylar için doktor kartının altındaki adres kartını bul
        # Her doktor kartına ait benzersiz ID'yi bulalım ve sonra bu ID'ye göre adres bilgilerini alalım
        try:
            # Doktor ID'sini profil URL'sinden ya da başka bir kaynaktan çıkaralım
            doctor_id = None
            
            # Doktor kartının hemen altındaki adres kartını bulmaya çalışalım
            # Önce doktor kartının parent elementini bulalım
            parent = card.parent
            if parent:
                # Doktor kartından sonraki elementi bulalım
                next_element = parent.find_next('div', class_='doctor-card-address')
                if next_element:
                    # Adres metni
                    address_text = next_element.find('span', class_='text-truncate')
                    if address_text:
                        doctor_info['address'] = address_text.text.strip()
                    
                    # Google harita linki
                    map_link = next_element.find('a', attrs={'data-test-id': 'address-map-link'})
                    if map_link:
                        doctor_info['map_link'] = map_link.get('href')
                    
                    # Kurumsal isim (eğer varsa)
                    entity_name = next_element.find('p', attrs={'data-test-id': 'entity-name'})
                    if entity_name:
                        doctor_info['institution'] = entity_name.text.strip()
                    
                    # İl ve ilçe bilgileri (meta tag'lerden)
                    city = next_element.find('meta', attrs={'data-test-id': 'city-address'})
                    province = next_element.find('meta', attrs={'data-test-id': 'province-address'})
                    if city and province:
                        doctor_info['location'] = f"{city.get('content')}, {province.get('content')}"
                    
                    # Online hizmet bilgisi
                    online_service = next_element.find('a', class_='nav-link', text=lambda t: t and 'Online' in t)
                    doctor_info['online_available'] = bool(online_service)
        except Exception as e:
            print(f"Adres bilgisi alınırken hata oluştu: {e}")
            
        doctors_list.append(doctor_info)
    
    return doctors_list

def get_doctors_from_website(url):
    """
    Belirtilen URL'den doktor bilgilerini alır
    
    Args:
        url (str): Doktor bilgilerinin yer aldığı web sayfası URL'si
        
    Returns:
        list: Doktor bilgilerinin listesi
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP hatalarını kontrol et
        
        return scrape_doctor_info(response.text)
    
    except requests.RequestException as e:
        print(f"Hata oluştu: {e}")
        return []

def get_doctors_from_html_file(file_path):
    """
    Belirtilen HTML dosyasından doktor bilgilerini alır
    
    Args:
        file_path (str): HTML dosyasının yolu
        
    Returns:
        list: Doktor bilgilerinin listesi
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        return scrape_doctor_info(html_content)
    
    except Exception as e:
        print(f"Dosya okunurken hata oluştu: {e}")
        return []

def get_doctors_from_url_with_pagination(base_url, max_pages=5):
    """
    Sayfalama yaparak tüm doktorların bilgilerini alır
    
    Args:
        base_url (str): Doktor bilgilerinin yer aldığı temel URL
        max_pages (int): Kontrol edilecek maksimum sayfa sayısı
        
    Returns:
        list: Tüm doktorların bilgilerinin olduğu liste
    """
    all_doctors = []
    
    for page in range(1, max_pages + 1):
        page_url = f"{base_url}?page={page}"
        print(f"Sayfa {page} aranıyor: {page_url}")
        
        doctors = get_doctors_from_website(page_url)
        if not doctors:
            break
            
        all_doctors.extend(doctors)
        print(f"Sayfa {page}: {len(doctors)} doktor bulundu")
    
    return all_doctors

def get_individual_doctor_info(url):
    """
    Belirtilen URL'den tek bir doktorun detaylı bilgilerini alır
    
    Args:
        url (str): Doktor profil sayfasının URL'si
        
    Returns:
        dict: Doktor bilgilerini içeren sözlük
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        doctor_info = {}
        
        # Doktor adı
        name_element = soup.find('h1', class_='doctor-profile-name')
        if name_element:
            doctor_info['name'] = name_element.text.strip()
        
        # Profil resmi
        img_element = soup.find('img', class_='doctor-profile-avatar')
        if img_element:
            doctor_info['profile_image'] = img_element.get('src')
        
        # Adres bilgileri - her bir adres bloğunu ayrı ayrı alalım
        address_blocks = soup.find_all('div', class_='doctor-card-address')
        
        addresses = []
        for block in address_blocks:
            address_info = {}
            
            # Adres metni
            address_text = block.find('span', class_='text-truncate')
            if address_text:
                address_info['full_address'] = address_text.text.strip()
            
            # Google harita linki
            map_link = block.find('a', attrs={'data-test-id': 'address-map-link'})
            if map_link:
                address_info['map_link'] = map_link.get('href')
            
            # Kurumsal isim
            entity_name = block.find('p', attrs={'data-test-id': 'entity-name'})
            if entity_name:
                address_info['institution'] = entity_name.text.strip()
            
            # Online hizmet bilgisi
            online_tab = block.find('a', class_='nav-link', text=lambda t: t and 'Online' in t)
            address_info['is_online'] = bool(online_tab)
            
            addresses.append(address_info)
        
        doctor_info['addresses'] = addresses
        
        return doctor_info
        
    except requests.RequestException as e:
        print(f"Hata oluştu: {e}")
        return {}

def scrape_many_doctors_directly_from_html(html_content):
    """
    Verilen HTML içeriğinden doğrudan doktor bilgilerini çıkarır
    Bu fonksiyon, sayfa yapısına daha uygun olarak her doktor için ilgili adres bilgisini bulmaya çalışır
    
    Args:
        html_content (str): HTML sayfası içeriği
        
    Returns:
        list: Doktor bilgilerini içeren sözlüklerin listesi
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    doctors_list = []
    
    # Tüm doktor kartlarını ve adres kartlarını bul
    doctor_sections = soup.find_all('div', class_='dp-doctor-listing-item')
    
    for section in doctor_sections:
        try:
            doctor_info = {}
            
            # Doktor kartı
            doctor_card = section.find('div', class_='dp-doctor-card')
            if not doctor_card:
                continue
                
            # Doktor adı
            name_element = doctor_card.find('span', attrs={'data-tracking-id': 'result-card-name'})
            if name_element:
                doctor_info['name'] = name_element.text.strip()
            
            # Profil resmi
            img_element = doctor_card.find('img')
            if img_element:
                doctor_info['profile_image'] = img_element.get('src')
            
            # Branş bilgisi
            specialization = doctor_card.find('span', attrs={'data-test-id': 'doctor-specializations'})
            if specialization:
                doctor_info['specialization'] = specialization.text.strip()
            
            # Değerlendirme sayısı
            reviews = doctor_card.find('span', class_='opinion-numeral')
            if reviews:
                doctor_info['reviews'] = reviews.text.strip()
            
            # Doktor onaylı profil mi?
            verified = doctor_card.find('span', attrs={'title': 'Teyit edilmiş profil'})
            doctor_info['verified_profile'] = bool(verified)
            
            # Profil URL'si
            profile_url = doctor_card.find('a', attrs={'data-id': 'address-context-cta'})
            if profile_url:
                doctor_info['profile_url'] = profile_url.get('href')
            
            # Adres kartı - doktor kartından sonraki element
            address_card = section.find('div', class_='doctor-card-address')
            if address_card:
                # Adres metni
                address_text = address_card.find('span', class_='text-truncate')
                if address_text:
                    doctor_info['address'] = address_text.text.strip()
                
                # Google harita linki
                map_link = address_card.find('a', attrs={'data-test-id': 'address-map-link'})
                if map_link:
                    doctor_info['map_link'] = map_link.get('href')
                
                # Kurumsal isim (eğer varsa)
                entity_name = address_card.find('p', attrs={'data-test-id': 'entity-name'})
                if entity_name:
                    doctor_info['institution'] = entity_name.text.strip()
                
                # İl ve ilçe bilgileri (meta tag'lerden)
                city = address_card.find('meta', attrs={'data-test-id': 'city-address'})
                province = address_card.find('meta', attrs={'data-test-id': 'province-address'})
                if city and province:
                    doctor_info['location'] = f"{city.get('content')}, {province.get('content')}"
                
                # Online hizmet bilgisi
                online_service = address_card.find('a', class_='nav-link', text=lambda t: t and 'Online' in t)
                doctor_info['online_available'] = bool(online_service)
            
            doctors_list.append(doctor_info)
            
        except Exception as e:
            print(f"Doktor bilgisi alınırken hata oluştu: {e}")
    
    return doctors_list

# Kullanım örneği
if __name__ == "__main__":
    # 1. Web sitesinden doktor bilgilerini çekme
    url = "https://www.doktortakvimi.com/ara?q=&loc=Ankara&page=2"
    doctors = get_doctors_from_website(url)
    
    # 2. Veya HTML dosyasından doktor bilgilerini çekme
    # doctors = get_doctors_from_html_file("doktor_sayfasi.html")
    
    # 3. Veya doktortakvimi.com'dan sayfalama yaparak tüm doktorların bilgilerini çekme
    # doctors = get_doctors_from_url_with_pagination("https://www.doktortakvimi.com/psikoloji/ankara")
    
    # 4. HTML içeriğinden doğrudan doktor bilgilerini çekme (örnek HTML içeriği burada olmalı)
    # html_content = """<div class="dp-doctor-listing-item">...</div>"""
    # doctors = scrape_many_doctors_directly_from_html(html_content)
    
    # Sonuçları DataFrame'e çevir ve yazdır
    if doctors:
        df = pd.DataFrame(doctors)
        # print(df)
        # CSV dosyasına kaydetme
        df.to_csv("doktorlar.csv", index=False, encoding='utf-8-sig')
        print(f"{len(doctors)} doktor bilgisi başarıyla kaydedildi.")
    else:
        print("Doktor bilgisi bulunamadı.")

20 doktor bilgisi başarıyla kaydedildi.


/tmp/ipykernel_194708/3810543517.py:88: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  online_service = next_element.find('a', class_='nav-link', text=lambda t: t and 'Online' in t)


In [14]:
pd.set_option("display.max_colwidth", None)

In [15]:
df

,name,profile_image,specialization,reviews,profile_url,verified_profile,address,map_link,institution,location,online_available
0,Uzm. Dt. Tansu Erakman,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/a97470/a97470ec0853f819d4bfe8bb02db51ff_medium_square.jpg,"Diş hekimi, Ağız diş ve çene cerrahisi",79\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/tansu-erakman/dis-hekimi-agiz-dis-ve-cene-cerrahisi/ankara,True,"Kennedy Cad. 23/8 Kavaklıdere, Ankara","https://www.google.com/maps/search/?api=1&query=39.9093094,32.8599739",Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False
1,Uzm. Kl. Psk. Armağan Gülenay,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/5e035c/5e035ccad246bd4f414e7b9c3c5f6b7c_medium_square.jpg,Psikoloji,16\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/armagan-gulenay/psikoloji/ankara,True,"Mustafa Kemal Mahallesi 2131. Sokak No:5/6 Poyraz Apartmanı, Çankaya","https://www.google.com/maps/search/?api=1&query=39.9135513,32.7684479",Tardu Kişisel Gelişim Eğitim ve Danışmanlık Hizmetleri,"Çankaya, Ankara",False
2,Uzm. Kl. Psk. Lütfi Atabey,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/186948/18694866b6ae42019bd8a47e94dffd42_medium_square.jpg,"Psikoloji, Psikolojik danışma ve rehberlik",24\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/lutfi-atabey/psikoloji-psikolojik-danisma-ve-rehberlik/ankara,True,"Çukurambar Mahallesi 1446. Sokak No:12/25 Alternatif Plaza, Ankara","https://www.google.com/maps/search/?api=1&query=39.906414,32.8108559",Uzm. Kl. Psk. Lütfi Atabey,"Ankara, Ankara",False
3,Psk. Serra Ekin Sönmez,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/95b23b/95b23bca53315a79d35db5c82e361c8b_medium_square.jpg,Psikoloji,40\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/serra-ekin-sonmez-2/psikoloji/ankara,True,"Beştepe Mah. Nergiz Sk. 7-2 Via Flat İş Merkezi Armada Kat:3 İç Kapı: 87 Söğütözü/Ankara, Ankara","https://www.google.com/maps/search/?api=1&query=39.9140816,32.8084946",Serra Psikolojik Danışmanlık,"Ankara, Ankara",False
4,Prof. Dr. Nuran Allı,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/b51759/b5175995e1a11f7d6c245f84f359287c_medium_square.jpg,Dermatoloji,16\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/nuran-alli/dermatoloji/ankara,True,"Ümit Mahallesi 2432.Cad Çamlıca Bulvar Sitesi D Blok 58/6, Ankara","https://www.google.com/maps/search/?api=1&query=39.8961067,32.7024689",Prof. Dr. Nuran Allı Muayenehanesi,"Ankara, Ankara",False
5,Dr. Muhammet Özgehan,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/d0f41f/d0f41fea60fd42fc6475d7838e6c42ae_medium_square.jpg,Sertifikalı medikal estetik,41\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/muhammet-ozgehan-2/sertifikali-medikal-estetik/ankara,True,"Ahmet Taner Kışlalı mah. 2800. sok. İlko Sitesi No: 7 Çayyolu, Ankara","https://www.google.com/maps/search/?api=1&query=39.8654442,32.6904526",TRİPAESTHETİC,"Ankara, Ankara",False
6,Op. Dr. Hilal Özer,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/9d24f2/9d24f2611ba54ec49a4c5eb89882efa3_medium_square.jpg,Genel cerrahi,8\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/hilal-ozer/genel-cerrahi/ankara,True,"Beştepe Mahallesi Yaşam Caddesi No:13A D:2 Neorama Plaza, Ankara","https://www.google.com/maps/search/?api=1&query=39.9139442,32.8054504",Hilal Özer Muayenehanesi,"Ankara, Ankara",False
7,Prof. Dr. Mustafa Benekli,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/69d921/69d9216774c4da566b77d6d554e1bb5f_medium_square.jpg,Tıbbi onkoloji,64\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/mustafa-benekli/tibbi-onkoloji/ankara,True,"1441. Cadde. MEVA İş Merkezi. No 8/20, Çukurambar, Ankara","https://www.google.com/maps/search/?api=1&query=39.9048157,32.8088608",Prof. Dr. Mustafa Benekli Muayenehanesi,"Ankara, Ankara",False
8,Uzm. Psk. Asya Mirza,//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/e8d488/e8d48823d6ff937ae9bf9b91099164dc_medium_square.jpg,Psikoloji,23\n\t\t\t\t\t\t\tgörüş,https://www.doktortakvimi.com/asya-mirza/psik

In [8]:
import pandas as pd
pd.DataFrame(doctors)

,name,profile_image,specialization,reviews,address,map_link,institution,location,online_available,profile_url,verified_profile
0,Uzm. Dt. Tansu Erakman,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,"Diş hekimi, Ağız diş ve çene cerrahisi",79\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/tansu-erakman/di...,True
1,Uzm. Kl. Psk. Armağan Gülenay,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Psikoloji,16\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/armagan-gulenay/...,True
2,Uzm. Kl. Psk. Lütfi Atabey,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,"Psikoloji, Psikolojik danışma ve rehberlik",24\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/lutfi-atabey/psi...,True
3,Psk. Serra Ekin Sönmez,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Psikoloji,40\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/serra-ekin-sonme...,True
4,Prof. Dr. Nuran Allı,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Dermatoloji,16\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/nuran-alli/derma...,True
5,Dr. Muhammet Özgehan,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Sertifikalı medikal estetik,41\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/muhammet-ozgehan...,True
6,Op. Dr. Hilal Özer,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Genel cerrahi,8\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/hilal-ozer/genel...,True
7,Prof. Dr. Mustafa Benekli,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Tıbbi onkoloji,64\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/mustafa-benekli/...,True
8,Uzm. Psk. Asya Mirza,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,Psikoloji,23\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/asya-mirza/psiko...,True
9,Psk. Safiye Elif Çam Köseoğlu,//s3-eu-west-1.amazonaws.com/doktortakvimi.com...,"Psikoloji, Aile danışmanlığı",12\n\t\t\t\t\t\t\tgörüş,"Kennedy Cad. 23/8 Kavaklıdere, Ankara",https://www.google.com/maps/search/?api=1&quer...,Uzm. Dr. Tansu ERAKMAN Muayehanesi,"Ankara, Ankara",False,https://www.doktortakvimi.com/safiye-elif-cam-...,True


In [6]:
import requests
import os
import re

def download_image(image_url, save_dir="indirilen_resimler"):
    """
    Belirtilen URL'den resmi indirir ve orijinal adıyla kaydeder
    
    Args:
        image_url (str): İndirilecek resmin URL'si
        save_dir (str): Resmin kaydedileceği klasör
    
    Returns:
        bool: İndirme başarılı olduğunda True, aksi halde False
    """
    # Göreli URL ise mutlak URL'ye çevir
    if image_url.startswith('//'):
        image_url = 'https:' + image_url
    
    try:
        # Resim adını URL'den çıkar
        file_name = os.path.basename(image_url)
        
        # Eğer URL'nin sonunda parametreler varsa, onları temizle
        file_name = re.sub(r'\?.*$', '', file_name)
        
        # Klasörün var olduğundan emin ol
        os.makedirs(save_dir, exist_ok=True)
        
        # Tam dosya yolu
        save_path = os.path.join(save_dir, file_name)
        
        # İsteği gönder
        response = requests.get(image_url)
        response.raise_for_status()  # HTTP hatalarını kontrol et
        
        # Dosyayı kaydet
        with open(save_path, 'wb') as file:
            file.write(response.content)
        
        print(f"Resim başarıyla indirildi: {save_path}")
        return True
    
    except requests.RequestException as e:
        print(f"Resim indirilirken hata oluştu: {e}")
        return False
    except IOError as e:
        print(f"Dosya kaydedilirken hata oluştu: {e}")
        return False

# Kullanım örneği
if __name__ == "__main__":
    # İndirilecek resim URL'si
    image_url = "//s3-eu-west-1.amazonaws.com/doktortakvimi.com/doctor/a86c0e/a86c0e2c0713a8d0c5eb3815c1d67be0_medium_square.jpg"
    
    download_image(image_url)

Resim başarıyla indirildi: indirilen_resimler/a86c0e2c0713a8d0c5eb3815c1d67be0_medium_square.jpg
